In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
from torch.utils.data import random_split, DataLoader, Dataset
from IPython.display import clear_output, display
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import scanpy as sc
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import pyensembl
import time

In [ ]:
import src.updatedmodel as m
import src.data_cleaning as dc
import src.position_encoding as pos
import src.pathway_encoding as path
import src.updatetraining as train

In [ ]:
ensembl_data = pyensembl.EnsemblRelease(109)
ensembl_data.download()
ensembl_data.index()

In [ ]:
adata = sc.read_h5ad('vcc_data/adata_Training.h5ad')
#adata.obs['perturbation_idx'] = np.random.randint(0, CONFIG["n_perturbations"], size=adata.n_obs)
gene_names = pd.read_csv('vcc_data/gene_names.csv', header = None)
gene_names = gene_names.iloc[:, 0].tolist()


CONFIG = {
    "n_genes": len(gene_names), #total number of expression genes needed to predict
    "n_perturbations": len(adata.obs['target_gene'].unique().tolist()), #number of unique perturbations
    "n_chromosomes": 25, #chromosome number 23+X+Y

    "perturbation_dim": 256,      # Condition embedding
    "chrom_embedding_dim": 16,     # Learned in-model for chromosome identity
    "locus_fourier_features": 8,   # Number of Fourier frequency pairs (2*F)
    "pathway_dim": 50,             # From pre-trained Autoencoder(based on hallmark MSigDB)
    "gene_identity_dim": 189,       # Main learnable gene embedding

    # Backbone dims
    "d_model": 512,                # Mamba hidden size
    "mamba_layers": 4,
    "n_heads": 8,
    "n_layers": 4,

    # Head
    "prediction_head": "probabilistic", # "linear" | "probabilistic"

    # Training
    "batch_size": 16,
    "learning_rate": 5e-5, # Lowered LR for AdamW stability
    "epochs": 100,
}

# Derived dimensions for clarity
# Positional encoding dim = chromosome embedding + MLP output from Fourier features
POS_DIM = CONFIG["chrom_embedding_dim"] + CONFIG["chrom_embedding_dim"]
GENE_FEAT_DIM = CONFIG["gene_identity_dim"] + CONFIG["pathway_dim"] + POS_DIM

In [ ]:
unique_genes = adata.var.index.tolist()
perturbation_to_idx_map = {name: i for i, name in enumerate(unique_genes)}
adata.obs['perturbation_idx'] = adata.obs['target_gene'].map(perturbation_to_idx_map)
adata.obs['perturbation_idx'] = adata.obs['perturbation_idx'].fillna(CONFIG['n_genes']).astype(int)
control_adata = dc.get_control_data(adata)
chr_idx, locus_norm, locus_fourier = pos.precompute_positional_indices(ensembl_data, gene_names, CONFIG)
pathway_feats = np.load('pathway_matrix.npy')

In [ ]:
device = torch.device("cuda")
print(f"\nUsing device: {device}")
model = m.TranscriptomePredictor(CONFIG, GENE_FEAT_DIM, pathway_feats, chr_idx, locus_fourier)
model.to(device)

dataset = m.PerturbationDataset(adata, CONFIG) # Pass CONFIG to the dataset constructor
train_loader = DataLoader(dataset, batch_size=CONFIG["batch_size"], shuffle=True)

In [ ]:
print("\n--- Checking every dataset entry for NaNs ---")

nan_found_at_index = -1
total_samples = len(dataset)

# Use tqdm for a progress bar
for i in tqdm(range(total_samples), desc="Scanning dataset"):
    sample = dataset[i]
    target_expr = sample["target_expression"]
    
    # Check if this tensor has any NaN values
    if torch.isnan(target_expr).any():
        nan_found_at_index = i
        break # Stop as soon as we find one

# --- Report the result ---
if nan_found_at_index != -1:
    print("\n!!!!! CRITICAL: NaN DETECTED !!!!!")
    print(f"A NaN value was found in the 'target_expression' for sample index: {nan_found_at_index}")
    print("This is the likely cause of your CUDA error.")
else:
    print("\nOK: Scanned all {total_samples} samples. The dataset is 100% clean of NaNs.")

In [ ]:
loss_history = train.train_model(model, train_loader, CONFIG,device)